In [5]:
var Model = require("../model/model-magma-next.js")
var fs = require("fs")
var tf = require("../../jslib/textFile")

In [2]:
var option = JSON.parse(fs.readFileSync("../model/model-magma-next-opt.json","utf8"))
var 

var {
      model,
      parameters,
      updateCondition,
      constrain
    } = Model(option)

[ { MgN_beforeCrystallization: 85,
    growth_stoichiometry_orthopyroxene: 0.5,
    growth_stoichiometry_spinel: 0.05,
    MgN_beforeMixing: 80,
    mixing_stoichiometry_orthopyroxene: 0.5,
    mixing_stoichiometry_spinel: 0.05,
    log10_tau: 6 },
  { MgN_beforeCrystallization: 85,
    growth_stoichiometry_orthopyroxene: 0.5,
    growth_stoichiometry_spinel: 0.05,
    MgN_beforeMixing: 80,
    mixing_stoichiometry_orthopyroxene: 0.5,
    mixing_stoichiometry_spinel: 0.05,
    log10_tau: 6 },
  { MgN_beforeCrystallization: 85,
    growth_stoichiometry_orthopyroxene: 0.5,
    growth_stoichiometry_spinel: 0.05,
    MgN_beforeMixing: 80,
    mixing_stoichiometry_orthopyroxene: 0.5,
    mixing_stoichiometry_spinel: 0.05,
    log10_tau: 6 },
  { MgN_beforeCrystallization: 85,
    growth_stoichiometry_orthopyroxene: 0.5,
    growth_stoichiometry_spinel: 0.05,
    MgN_beforeMixing: 80,
    mixing_stoichiometry_orthopyroxene: 0.5,
    mixing_stoichiometry_spinel: 0.05,
    log10_tau: 6 },
  { 